## 1. Load the dataset

The dataset used in this example is survey data on food waste during ramadan
(provided by Jordan W-M)


To run this notebook, you will need to install: pandas, openai, transformers,
plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and
scipy.


In [14]:
import pandas as pd
from openai import OpenAI
client = OpenAI()


In [26]:
def get_embedding(input: str):
    response = client.embeddings.create(
      input=input,
      model="text-embedding-ada-002"
    )
    return response.data[0].embedding

In [29]:
# input_datapath = "data/english_free_text.csv"
# headers = ["whyRamLess", "whyCompMore", "whyProblemSoc", "encourage"]
# cols_to_embed = ["whyRamLess", "whyCompMore", "whyProblemSoc", "encourage"]
# index_col = 0

input_datapath = "data/gambling_feedback_data.csv"
headers = ["feedback","treatment","limit_on","block_on"]
cols_to_embed = ["feedback"]
index_col = None

In [21]:
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [25]:
# load & inspect dataset
df = pd.read_csv(input_datapath, index_col=index_col)
df = df[headers]

df = df.dropna()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df.head(2)


/var/folders/4c/5kctcgk55539047v8xy3clj80000gn/T/ipykernel_91074/3179908913.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,feedback,treatment,limit_on,block_on
0,good,4,0,0
1,no it was helpful,4,0,0


In [13]:
# # subsample to 1k most recent reviews and remove samples that are too long
# top_n = 1000
# # first cut to first 2k entries, assuming less than half will be filtered out
# df = df.tail(top_n * 2)

# encoding = tiktoken.get_encoding(embedding_encoding)

# questionId = 0  # the first question

# # omit reviews that are too long to embed
# df["n_tokens"] = df.iloc[:, questionId].apply(lambda x: len(encoding.encode(x)))

# df = df[df.n_tokens <= max_tokens].tail(top_n)
# len(df)


## 2. Get embeddings and save them for future reuse


In [30]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
for col in cols_to_embed:
    print(f'Getting embeddings for column "{col}"')
    df[col + "-embedding"] = df.loc[:, col].apply(get_embedding)


df.to_csv(f"{input_datapath.split('.')[0]}_with_embeddings.csv")


Getting embeddings for column "feedback"
